### Self attention solves a major issue in RNN which faces unable to process large/long text/docs. RNN also suffers from vanishinng gradients and gradient explosions, often needs large training step to reach local/global minima
